In [0]:
import os
import json
import fnmatch
import cv2
import numpy as np
import string
import time
import pdb;
from time import time
import glob, shutil
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.preprocessing.sequence import pad_sequences
from matplotlib import pyplot as plt
from keras.layers import SeparableConv2D,ReLU,CuDNNGRU, add,MaxPooling2D , Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional,Activation
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,TensorBoard,EarlyStopping
from keras.layers.recurrent import GRU
import tensorflow as tf
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'

### Import all necessary  library


### Mount Google Drive to Colab

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
%cd /content/drive/'My Drive/Vietnamese'

/content/drive/My Drive/Vietnamese


In [0]:
import tensorflow as tf

#ignore warnings in the output
tf.logging.set_verbosity(tf.logging.ERROR)

In [35]:
from tensorflow.python.client import device_lib

# Check all available devices if GPU is available
print(device_lib.list_local_devices())
TF_CONFIG_=tf.ConfigProto()
TF_CONFIG_.allow_soft_placement=True
TF_CONFIG_.gpu_options.per_process_gpu_memory_fraction=0.4
TF_CONFIG_.log_device_placement=True
sess = tf.Session(config=TF_CONFIG_)
# sess.run(init)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13396373273913228708
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2997015954750289465
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9614792480319192142
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11326753997
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15739511965698922232
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [36]:
char_list=' !"#&\'()*+,-./0123456789:;?AÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶBCDĐEÉÈẺẼẸÊẾỀỂỄỆFGHIÍÌỈĨỊJKLMNOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢPQRSTUÚÙỦŨỤỨỪỬỮỰVWXYÝỲỶỸỴZaáàảãạâấầẩẫậăắằẳẵặbcdđeéèẻẽẹêếềểễệfghiíìỉĩịjklmnoóòỏõọôốồổỗộơớờởỡợpqrstuúùủũụưứừửữựvwxyýỳỷỹỵz'
print(char_list)

 !"#&'()*+,-./0123456789:;?AÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶBCDĐEÉÈẺẼẸÊẾỀỂỄỆFGHIÍÌỈĨỊJKLMNOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢPQRSTUÚÙỦŨỤỨỪỬỮỰVWXYÝỲỶỸỴZaáàảãạâấầẩẫậăắằẳẵặbcdđeéèẻẽẹêếềểễệfghiíìỉĩịjklmnoóòỏõọôốồổỗộơớờởỡợpqrstuúùủũụưứừửữựvwxyýỳỷỹỵz


### Encoding the Label to number array

> By the Charmap above include the Latin, Vietnamese character and symbol (with the **Blank** too): 

!"#&\'()*+,-./0123456789:;?AÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶBCDĐEÉÈẺẼẸÊẾỀỂỄỆFGHIÍÌỈĨỊJKLMNOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢPQRSTUÚÙỦŨỤỨỪỬỮỰVWXYÝỲỶỸỴZaáàảãạâấầẩẫậăắằẳẵặbcdđeéèẻẽẹêếềểễệfghiíìỉĩịjklmnoóòỏõọôốồổỗộơớờởỡợpqrstuúùủũụưứừửữựvwxyýỳỷỹỵz


In [0]:
def encode_to_labels(txt):
    # encoding each output word into digits
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
#             print(dig_lst)
        except:
            print(char)
        
    return dig_lst

### Preprocessing image
First I generated the data with this tool that I customize a little in [this repo](https://github.com/moto8xpk/TextRecognitionDataGenerator)
* Before this, I use some script to modify the size of data to (280,32)

Use opencv to read image and convert to black white image


In [0]:
# lists for training dataset
training_img = []
training_txt = []
train_input_length = []
train_label_length = []
orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []
 
max_label_len = 0

i =1 
flag = 0


In [49]:
# for root , directory name and file name
for root, dirnames, filenames in os.walk('./src/training/out-pre'):
    # preprocessing image
    for f_name in fnmatch.filter(filenames, '*.jpg'):
        # read input image and convert into gray scale image
        stream = open(os.path.join(root, f_name),"rb")
        bytes = bytearray(stream.read())
        numpyarray = np.asarray(bytes, dtype=np.uint8)
        img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Convert gray scale image to black and white image
        (thresh, img) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
#         image_input_show = plt.imshow(img)  
#         plt.show()
        # convert each image of shape (32, 128, 1)
        w , h = img.shape
        
        if h > 280 or w > 32:
            continue
        if w < 32:
            add_zeros = np.ones((32-w, h))*255
            img = np.concatenate((img, add_zeros))
 
        if h < 280:
            add_zeros = np.ones((32,280-h))*255
            img = np.concatenate((img, add_zeros), axis=1)
        img = np.expand_dims(img , axis = 2)
        
        # Normalize each image
#         img = img/255.

        # get the text from the image
        txt = f_name.split('_')[0]

        # compute maximum length of the text
        if len(txt) > max_label_len:
            max_label_len = len(txt)
           
        # split the 10000 data into validation and training dataset as 10% and 90% respectively
        if i%4 == 0:     
            valid_orig_txt.append(txt)   
            valid_label_length.append(len(txt))
            valid_input_length.append(67)
            valid_img.append(img)
            valid_txt.append(encode_to_labels(txt))
        else:
            orig_txt.append(txt)   
            train_label_length.append(len(txt))
            train_input_length.append(67)
            training_img.append(img)
            training_txt.append(encode_to_labels(txt))
           
        # break the loop if total data is 15000
        if i == 15000:
            flag = 1
            break
        i+=1
    if flag == 1:
        break

print(len(training_img))
print(len(valid_img))

́
́
́
́
̀
́
́
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
́
̉
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
̀
́
̂
́
̂
́
̂
́
̂
́
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
̂
́
́
́
́
́
́
́
́
́
́
́
́
́
́
́
́
́
́
́
́
́
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
̂
́
̣
̂
8304
2768


In [0]:
# pad each output label to maximum text length
 
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

# input with shape of height=32 and width=280 
inputs = Input(shape=(32,280,1))


### Create The Architechtur of CRNN
This network architecture is inspired by this [paper](https://arxiv.org/pdf/1507.05717.pdf).

1.   Input shape for our architecture having an input image of height 32 and width 280. 
2.   Here we used 7 convolution layers of which 6 are having kernel size (3,3) and the last one is of size (2.2). And the number of filters is increased from 64 to 512 layer by layer.
3.   Two max-pooling layers are added with size (2,2) and then two max-pooling layers of size (2,1) are added to extract features with a larger width to predict long texts.
4.   Also, we used batch normalization layers after fifth and sixth convolution layers which accelerates the training process.
5.   Then we used a lambda function to squeeze the output from conv layer and make it compatible with GRU layer.
6.    Then used two Bidirectional GRU layers each of which has 128 units. This RNN layer gives the output of size (batch_size, 31, 221). Where 63 is the total number of output classes including blank character.

In [0]:

# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)


squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

# bidirectional GRU layers with units=128
blstm_1 = Bidirectional(GRU(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(GRU(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
# act_model = Model(inputs, outputs)

In [52]:
# act_model.summary()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 280, 1)   0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 32, 280, 64)  640         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_9 (MaxPooling2D)  (None, 16, 140, 64)  0           conv2d_15[0][0]                  
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 16, 140, 128) 73856       max_pooling2d_9[0][0]            
__________________________________________________________________________________________________
max_poolin


## Loss Function
Here, we are using the CTC loss function. CTC loss is very helpful in text recognition problems. It helps us to prevent annotating each time step and help us to get rid of the problem where a single character can span multiple time step which needs further processing if we do not use CTC. If you want to know more about CTC( Connectionist Temporal Classification ) please follow [this paper](https://www.cs.toronto.edu/~graves/icml_2006.pdf).

A CTC loss function requires four arguments to compute the loss, predicted outputs, ground truth labels, input sequence length to LSTM and ground truth label length. To get this we need to create a custom loss function and then pass it to the model. To make it compatible with our model, we will create a model which takes these four inputs and outputs the loss. This model will be used for training and for testing we will use the model that we have created earlier “act_model”.

In [0]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])



#model to be used at training time
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

In [54]:
print(inputs)
print(input_length)
print(labels)
print(label_length)

Tensor("input_3:0", shape=(?, 32, 280, 1), dtype=float32)
Tensor("input_length_2:0", shape=(?, 1), dtype=int64)
Tensor("the_labels_2:0", shape=(?, 29), dtype=float32)
Tensor("label_length_2:0", shape=(?, 1), dtype=int64)


## Configuration of the model

In [0]:
# opt=RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
# opt = Adagrad(lr=0.01, epsilon=None, decay=0.0)
opt= SGD(lr=0.001,decay=1e-6,momentum=0.9,nesterov=True,clipnorm=5)
# opt=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-9,
#                  decay=1e-9, amsgrad=True, clipnorm=5., clipvalue=0.5)
reduce_lr = ReduceLROnPlateau(patience=6, verbose=1, facttor=0.75)
# tensorboard=TensorBoard(log_dir='./logs-keras/{}'.format(time()), histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')
earlyStop=EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

In [0]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = opt,metrics=['accuracy'])
 
filepath="./best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint,reduce_lr,earlyStop]

In [0]:
training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

## Load your trained model to continue the training

In [0]:
# model.load_weights('./best_model.hdf5')

## Training Model

In [0]:
batch_size = 60
epochs = 50
model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length], 
          y=np.zeros(len(training_img)), 
          batch_size=batch_size, 
          epochs = epochs, 
          validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], 
          [np.zeros(len(valid_img))]), 
          verbose = 1, 
          callbacks = callbacks_list)

Train on 8304 samples, validate on 2768 samples
Epoch 1/50
8304/8304 [==============================] - 125s 15ms/step - loss: 54.3621 - acc: 0.0000e+00 - val_loss: 28.9075 - val_acc: 0.0000e+00

Epoch 00001: val_loss improved from inf to 28.90753, saving model to ./best_model.hdf5
Epoch 2/50
8304/8304 [==============================] - 115s 14ms/step - loss: 26.8784 - acc: 0.0000e+00 - val_loss: 29.0128 - val_acc: 0.0000e+00

Epoch 00002: val_loss did not improve from 28.90753
Epoch 3/50
8304/8304 [==============================] - 115s 14ms/step - loss: 22.4747 - acc: 0.0000e+00 - val_loss: 20.4261 - val_acc: 0.0000e+00

Epoch 00003: val_loss improved from 28.90753 to 20.42612, saving model to ./best_model.hdf5
Epoch 4/50
8304/8304 [==============================] - 115s 14ms/step - loss: 16.5019 - acc: 0.0000e+00 - val_loss: 14.6328 - val_acc: 0.0000e+00

Epoch 00004: val_loss improved from 20.42612 to 14.63278, saving model to ./best_model.hdf5
Epoch 5/50
1080/8304 [==>............

## Testing model

In [0]:
act_model = Model(inputs, outputs)
# # load the saved best model weights
act_model.load_weights('./best_model.hdf5')
# # predict outputs on validation images
prediction = act_model.predict(valid_img)
 
# # use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
# # see the results
i = 0
for x in out:
    print("image = ")
    print("original_text =  ", valid_orig_txt[i])
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')
    i+=1

## Infer the test
when I run the crop image srcipt this will be the Python script that I will create as a API service on my server

In [0]:
import cv2
import numpy as np
from keras import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import keras.backend as K
import tensorflow as tf
from keras.layers import SeparableConv2D,ReLU,CuDNNGRU, add,MaxPooling2D , Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional,Activation
from keras.layers.recurrent import GRU
from keras.activations import relu, sigmoid, softmax
from keras.models import Model
char_list=' !"#&\'()*+,-./0123456789:;?AÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶBCDĐEÉÈẺẼẸÊẾỀỂỄỆFGHIÍÌỈĨỊJKLMNOÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢPQRSTUÚÙỦŨỤỨỪỬỮỰVWXYÝỲỶỸỴZaáàảãạâấầẩẫậăắằẳẵặbcdđeéèẻẽẹêếềểễệfghiíìỉĩịjklmnoóòỏõọôốồổỗộơớờởỡợpqrstuúùủũụưứừửữựvwxyýỳỷỹỵz'

inputs = Input(shape=(32,280,1))
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)

# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)


squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(GRU(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(GRU(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(212+1, activation = 'softmax')(blstm_2) #char list len is 212

inference_model=Model(inputs, outputs)
inference_model.load_weights('./best_model.hdf5')

stream = open('./img37.png',"rb")
bytes = bytearray(stream.read())
numpyarray = np.asarray(bytes, dtype=np.uint8)
img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
(thresh, img) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
# cv2.imshow('image',img)
image_input_show = plt.imshow(img)  
plt.show()
w , h = img.shape
if w < 32:
    add_zeros = np.ones((32-w, h))*255
    img = np.concatenate((img, add_zeros))

if h < 280:
    add_zeros = np.ones((32,280-h))*255
    img = np.concatenate((img, add_zeros), axis=1)
img = np.expand_dims(img , axis = 2)
img = np.expand_dims(img , axis = 0)
print(img.shape)
# img = img/255.0

# print(img)

preds = inference_model.predict(img)
out = K.get_value(K.ctc_decode(preds, input_length=np.ones(preds.shape[0])*preds.shape[1],
                         greedy=True)[0][0])
i = 0
for x in out:
  # print("original_text =  ", valid_orig_txt[i])
  print("predicted text = ", end = '')
  for p in x:  
      if int(p) != -1:
          print(char_list[int(p)], end = '')       
  print('\n')
  i+=1

# Conclusion


1.   First of all, this was the first project that I have learn about the text-based extraction so I have a lot of problem when I deal with dataset and create architecture of the neural network of Convolution Recurrent neural network
2.   In my project, the dataset I use for training is not suitable for extraction data of the Medical card because  my model predict pretty well with my data that I generated from the tool
3.   In further, I will test some VGG16 or Resnet to estimate the accuracy of the model and I will collect more data of the real form of medical record to extract.

